## Initialization

In [1]:
from active_learning.al import AL
from utils.utils import get_tasks

# exp selection
exp_name = 'DFFC_8D_4'

## Save and load

In [3]:
# load al instance from database
al = AL(exp_name)

In [ ]:
# save the current al instance to the database
al.save_exp_to_db()

## AL main workflow

### recipe generation

In [ ]:
%%time
# generate next trial using Bayesian Optimization, specify beta if needed, e.g. beta=100 for exploration, beta=0.2 for default
al.generate_botorch_trial(n=36, aquisition = 'qUCB', optimizer_kwargs = {'options':{'maxiter':20000000}, 'sequential':True})

In [ ]:
# generate next trial using SOBOL method (random select)
al.generate_sobol_trial(n=20)

In [ ]:
# update the recipe table on the database, if trial index is not specified, the recipe of the latest trial will be uploaded
al.update_recipe_table(trial_index=None, add_pred=True)

### sample preparation

In [ ]:
# benchmark specification
benchmark = None
# benchmark={5: '1_0'}

In [ ]:
# command opentrons to prepare the sample for the latest trial, following the ot2_run_configs of the exp
al.make_sample(trial_index=None, benchmark=benchmark, operator_name='chu')

In [ ]:
# update the sample table on the database, if trial index is not specified, the sample of the latest trial will be uploaded. Note that sample table calculation is based on local ot2_run_configs.py under project folder, not the one on ot2 server.
al.update_sample_table(trial_index=None, benchmark=benchmark)

### sample testing complete

In [ ]:
# run this cell after sample testing and data analysis is completed, then go back to the BO step
al.mark_trial_complete(trial_index=None)

## AL monitor

In [ ]:
# update the exp with the latest result
al.update_exp_result()

In [ ]:
# exp monitor for windows
al.exp_monitor()

In [ ]:
# exp monitor for mac and linux
from IPython.display import display, HTML
display(HTML(al.exp_monitor().to_html()))

In [ ]:
# check abandon reason
al.exp.trials[0].abandoned_reason

## AL manual edit

In [ ]:
# update the exp with the latest result and force match previous trials with the results in the database
al.update_exp_result(force_update_trial=[0, 1])

In [ ]:
# manually add a trial
trial_df = get_tasks(exp_name, task_name='manual')
al.add_manual_trial(trial_df)

In [ ]:
# abandon an certain arm in a trial
al.abandon_arm(0, '0_18', 'abandoned in manual test')

In [ ]:
# abandon a whole trial
al.abandon_trial(9, 'test trial')

## Plotting

In [ ]:
from ax.modelbridge.cross_validation import cross_validate
from ax.plot.contour import interact_contour
from ax.plot.diagnostic import interact_cross_validation
from ax.plot.scatter import interact_fitted
from ax.plot.slice import interact_slice
from ax.utils.notebook.plotting import render, init_notebook_plotting
init_notebook_plotting()

In [ ]:
model = al.get_bo_model()

In [ ]:
# Contour plots
render(interact_contour(model=model, metric_name='max_power'))

In [ ]:
# Cross-validation plots
cv_results = cross_validate(model)
print('current r_squared is: ', al.calculate_r_squared(cv_results))
render(interact_cross_validation(cv_results))

In [ ]:
# Slice plots
render(interact_slice(model))

In [ ]:
# Tile plots
render(interact_fitted(model, rel=False))

## Danger Zone

In [ ]:
# delete the current active learning process from the database
al.delete_exp_on_db()

In [ ]:
# only use this if you want to start a new experiment, note that the previous experiment has to be manually deleted, first, because the script refuses to overwrite the previous experiment by default
al = AL(exp_name, load_from_db=False)